# MFIT5010 Final Project Report

## CHEN, Zhuohui 20657689

In [33]:
import numpy as np
import pandas as pd
from subprocess import check_output
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

### First, importing the data from kaggle. 

In [34]:
# import data
print('Loading data...')
data_path = "/Users/solstice/Downloads/kkbox-music-recommendation-challenge/"
df_test = pd.read_csv(data_path + 'test.csv',dtype={'msno' : 'category','source_system_tab' : 'category','source_screen_name' : 'category','source_type' : 'category','song_id' : 'category'})
df_train = pd.read_csv(data_path + 'train.csv',dtype={'msno' : 'category','source_system_tab' : 'category','source_screen_name' : 'category','source_type' : 'category','target' : np.uint8,'song_id' : 'category'})
df_members = pd.read_csv(data_path + 'members.csv',dtype={'city' : 'category','bd' : 'category','gender' : 'category','registered_via' : 'category'})
df_songs = pd.read_csv(data_path + 'songs.csv',dtype={'genre_ids': 'category','language' : 'category','artist_name' : 'category','composer' : 'category','lyricist' : 'category','song_id' : 'category'})

### Then, we need to process the raw data, especially those features related to time.

In [35]:
print('Data preprocessing...')

#deal with 'time' in members 
df_members['registration_year'] = df_members['registration_init_time'].apply(lambda x : int(str(x)[0:4]))
df_members['registration_year'] = pd.to_numeric(df_members['registration_year'],downcast='unsigned')
df_members['registration_month'] = df_members['registration_init_time'].apply(lambda x : int(str(x)[4:6]))
df_members['registration_month'] = pd.to_numeric(df_members['registration_month'],downcast='unsigned')
df_members['registration_date'] = df_members['registration_init_time'].apply(lambda x : int(str(x)[6:8]))
df_members['registration_date'] = pd.to_numeric(df_members['registration_date'],downcast='unsigned')
df_members.drop('registration_init_time',axis=1,inplace=True)
df_members['expiration_year'] = df_members['expiration_date'].apply(lambda x : int(str(x)[0:4]))
df_members['expiration_year'] = pd.to_numeric(df_members['expiration_year'],downcast='unsigned')
df_members['expiration_month'] = df_members['expiration_date'].apply(lambda x :int(str(x)[4:6]))
df_members['expiration_month'] = pd.to_numeric(df_members['expiration_month'],downcast='unsigned')
df_members['expiration_date'] = df_members['expiration_date'].apply(lambda x : int(str(x)[6:8]))
df_members['expiration_date'] = pd.to_numeric(df_members['expiration_date'],downcast='unsigned')

In [47]:
df_members.columns

Index(['msno', 'city', 'bd', 'gender', 'registered_via', 'expiration_date',
       'registration_year', 'registration_month', 'registration_date',
       'expiration_year', 'expiration_month'],
      dtype='object')

### After processing data, we can generate the final training data set and testing data set as following.

In [37]:
df_test = pd.merge(left = df_test,right = df_members,how='left',on='msno')
df_test.msno = df_test.msno.astype('category')
df_train = pd.merge(left = df_train,right = df_members,how='left',on='msno')
df_train.msno = df_train.msno.astype('category')

In [19]:
df_test = pd.merge(left = df_test,right = df_songs,how = 'left',on='song_id')
df_test.song_length.fillna(200000,inplace=True)
df_test.song_length = df_test.song_length.astype(np.uint32)
df_test.song_id = df_test.song_id.astype('category')
df_train = pd.merge(left = df_train,right = df_songs,how = 'left',on='song_id')
df_train.song_length.fillna(200000,inplace=True)
df_train.song_length = df_train.song_length.astype(np.uint32)
df_train.song_id = df_train.song_id.astype('category')

### Let us have a quick look on the training data set and testing data set.

In [49]:
df_train

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,bd,gender,registered_via,expiration_date,registration_year,registration_month,registration_date,expiration_year,expiration_month
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,1,0,NaN,7,5,2012,1,2,2017,10
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,13,24,female,9,11,2011,5,25,2017,9
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,13,24,female,9,11,2011,5,25,2017,9
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,13,24,female,9,11,2011,5,25,2017,9
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,1,0,NaN,7,5,2012,1,2,2017,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377413,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,my library,Local playlist more,local-playlist,1,4,26,female,9,30,2004,12,18,2017,9
7377414,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,search,Search,song,0,1,0,NaN,7,6,2013,6,27,2017,10
7377415,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,search,Search,song,1,1,0,NaN,7,6,2013,6,27,2017,10
7377416,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,discover,Discover Genre,online-playlist,1,5,0,NaN,9,18,2015,2,13,2017,9


### Notice that in the test set, we do not have the 'targer' column, that is what we need to predict.

In [50]:
df_test

,id,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,expiration_date,registration_year,registration_month,registration_date,expiration_year,expiration_month
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library,1,0,NaN,7,18,2016,2,19,2017,9
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library,1,0,NaN,7,18,2016,2,19,2017,9
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist,1,0,NaN,4,24,2016,11,17,2016,11
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio,3,30,male,9,30,2007,7,25,2017,4
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio,3,30,male,9,30,2007,7,25,2017,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2556785,2556785,XmA/cIkXJ8jZUfsUc4bBfJVWMMqmylnPW0WVkg/iz0s=,wJoWzZd7AL+qX9xZWZwRTzGRYg0Lxcl5Pe+9n5hZgAQ=,discover,Online playlist more,online-playlist,13,41,female,9,21,2010,11,26,2017,9
2556786,2556786,jvbujcxCExG0CrPShsEmZ6pePcHuRqru2OLEUw85iGk=,KCJ8BlSfRQRgB7EVuzFvg52AhR8m2fT032MzN5ewbEI=,discover,NaN,online-playlist,13,24,male,9,12,2014,5,6,2017,9
2556787,2556787,jvbujcxCExG0CrPShsEmZ6pePcHuRqru2OLEUw85iGk=,ySDsKJSnhFMESzC9mBhY0hnFEEBWDYx0FOMNcYDLb/A=,discover,NaN,online-playlist,13,24,male,9,12,2014,5,6,2017,9
2556788,2556788,jvbujcxCExG0CrPShsEmZ6pePcHuRqru2OLEUw85iGk=,WXH8kL8e+0H2jY+s2Y6FpHyyHdaMYV3b6yfDa3FAt9U=,discover,NaN,online-playlist,13,24,male,9,12,2014,5,6,2017,9


### This project decides to use LightGBM algorithm to train the model, predicting whether users will listen to a song repeatedly within a month, as a personalized recommendation goal. LightGBM is a gradient boosting framework that uses tree based learning algorithms.

### What's more, in order to avoid the probability of overfitting on the training set, I decide to apply the K-Fold methods, with decision of K equal to 3, because 5 or more is too time-consuming for my personal laptop and 3 might be a suitable number. Instead of randomly deviding the trainning data set into train set and validation set, K-fold method repeats this process K times and make K predictions. By averaging the K predictions, we can efficiently avoid the probability of overfitting.

### After some experiment of tuning parameters, I found tuning learning rate have more influence on the accuracy, so I start with learning rate of 0.2.

In [44]:
kf = KFold(n_splits=3)
predictions = np.zeros(shape=[len(df_test)])
print('Training LGBM model for the first time with learning rate 0.2...')
for train_indices,val_indices in kf.split(df_train) : 
    train_data = lgb.Dataset(df_train.drop(['target'],axis=1).loc[train_indices,:],label=df_train.loc[train_indices,'target'])
    val_data = lgb.Dataset(df_train.drop(['target'],axis=1).loc[val_indices,:],label=df_train.loc[val_indices,'target'])
    
    params = {}
    params['learning_rate'] = 0.2
    params['application'] = 'binary'
    params['max_depth'] = 8
    params['num_leaves'] = 2**8
    params['verbosity'] = 0
    params['metric'] = 'auc'
    
    model = lgb.train(params, train_data, 100, valid_sets=[val_data])
    predictions += model.predict(df_test.drop(['id'],axis=1))
    del model

Training LGBM model...
[1]	valid_0's auc: 0.683925
[2]	valid_0's auc: 0.691467
[3]	valid_0's auc: 0.698208
[4]	valid_0's auc: 0.700597
[5]	valid_0's auc: 0.708117
[6]	valid_0's auc: 0.710175
[7]	valid_0's auc: 0.711324
[8]	valid_0's auc: 0.714885
[9]	valid_0's auc: 0.716366
[10]	valid_0's auc: 0.71997
[11]	valid_0's auc: 0.720607
[12]	valid_0's auc: 0.721385
[13]	valid_0's auc: 0.722091
[14]	valid_0's auc: 0.722718
[15]	valid_0's auc: 0.723263
[16]	valid_0's auc: 0.724046
[17]	valid_0's auc: 0.724463
[18]	valid_0's auc: 0.725055
[19]	valid_0's auc: 0.729166
[20]	valid_0's auc: 0.730105
[21]	valid_0's auc: 0.73066
[22]	valid_0's auc: 0.732352
[23]	valid_0's auc: 0.732936
[24]	valid_0's auc: 0.73359
[25]	valid_0's auc: 0.73408
[26]	valid_0's auc: 0.734455
[27]	valid_0's auc: 0.734525
[28]	valid_0's auc: 0.734952
[29]	valid_0's auc: 0.735282
[30]	valid_0's auc: 0.735975
[31]	valid_0's auc: 0.73631
[32]	valid_0's auc: 0.73666
[33]	valid_0's auc: 0.736973
[34]	valid_0's auc: 0.737212
[35]	v

[85]	valid_0's auc: 0.649254
[86]	valid_0's auc: 0.64937
[87]	valid_0's auc: 0.649482
[88]	valid_0's auc: 0.649589
[89]	valid_0's auc: 0.649722
[90]	valid_0's auc: 0.649877
[91]	valid_0's auc: 0.649986
[92]	valid_0's auc: 0.650058
[93]	valid_0's auc: 0.650151
[94]	valid_0's auc: 0.650226
[95]	valid_0's auc: 0.650344
[96]	valid_0's auc: 0.650426
[97]	valid_0's auc: 0.650511
[98]	valid_0's auc: 0.650551
[99]	valid_0's auc: 0.650671
[100]	valid_0's auc: 0.650762


### Then, I increase the learning rate to 0.5, and compare the accuracy.

In [45]:
predictions2 = np.zeros(shape=[len(df_test)])
print('Training LGBM model for the first time with learning rate 0.5...')
for train_indices,val_indices in kf.split(df_train) : 
    train_data = lgb.Dataset(df_train.drop(['target'],axis=1).loc[train_indices,:],label=df_train.loc[train_indices,'target'])
    val_data = lgb.Dataset(df_train.drop(['target'],axis=1).loc[val_indices,:],label=df_train.loc[val_indices,'target'])
    
    params = {}
    params['learning_rate'] = 0.5
    params['application'] = 'binary'
    params['max_depth'] = 8
    params['num_leaves'] = 2**8
    params['verbosity'] = 0
    params['metric'] = 'auc'
    
    model = lgb.train(params, train_data, 100, valid_sets=[val_data])
    predictions2 += model.predict(df_test.drop(['id'],axis=1))
    del model

Training LGBM model for the first time with learning rate 0.5...
[1]	valid_0's auc: 0.683925
[2]	valid_0's auc: 0.696014
[3]	valid_0's auc: 0.701343
[4]	valid_0's auc: 0.705448
[5]	valid_0's auc: 0.712758
[6]	valid_0's auc: 0.715122
[7]	valid_0's auc: 0.717226
[8]	valid_0's auc: 0.719823
[9]	valid_0's auc: 0.722096
[10]	valid_0's auc: 0.722938
[11]	valid_0's auc: 0.72392
[12]	valid_0's auc: 0.72497
[13]	valid_0's auc: 0.726164
[14]	valid_0's auc: 0.727076
[15]	valid_0's auc: 0.727869
[16]	valid_0's auc: 0.728827
[17]	valid_0's auc: 0.729945
[18]	valid_0's auc: 0.730869
[19]	valid_0's auc: 0.73144
[20]	valid_0's auc: 0.732712
[21]	valid_0's auc: 0.734719
[22]	valid_0's auc: 0.735612
[23]	valid_0's auc: 0.736247
[24]	valid_0's auc: 0.736784
[25]	valid_0's auc: 0.737253
[26]	valid_0's auc: 0.737683
[27]	valid_0's auc: 0.738061
[28]	valid_0's auc: 0.738141
[29]	valid_0's auc: 0.738568
[30]	valid_0's auc: 0.738822
[31]	valid_0's auc: 0.738913
[32]	valid_0's auc: 0.740023
[33]	valid_0's auc:

[84]	valid_0's auc: 0.656085
[85]	valid_0's auc: 0.656104
[86]	valid_0's auc: 0.65626
[87]	valid_0's auc: 0.656441
[88]	valid_0's auc: 0.656559
[89]	valid_0's auc: 0.656616
[90]	valid_0's auc: 0.656602
[91]	valid_0's auc: 0.656656
[92]	valid_0's auc: 0.656699
[93]	valid_0's auc: 0.656836
[94]	valid_0's auc: 0.656866
[95]	valid_0's auc: 0.656982
[96]	valid_0's auc: 0.657103
[97]	valid_0's auc: 0.657066
[98]	valid_0's auc: 0.657105
[99]	valid_0's auc: 0.657165
[100]	valid_0's auc: 0.657263


### Comparing these two results of accuracy, we can find that the model result of learning rate of 0.5 has slightly better final AUC, which is 0.751234, 0.70523, 0.657263. So we chose the corresponding predictions in the test set as the final prediction and save them to the submission csv.

In [25]:
print('Making predictions and saving them...')
predictions2 = predictions2/3
submission = pd.read_csv(data_path + 'sample_submission.csv')
submission.target=predictions
submission.to_csv(data_path + 'submission.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')
print('Done!')